In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_trash = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [3]:
#Missed Trash Pickups
#The city is considering different ways to calculate fines. Here are two methods that are being considered. 
#Calculate the total fines that would be collected by each and compare that total to the amount collected using the current method.

#Alternative Method 1:
#For each address, if there are three or more missed pickups within a 180-day period, damages of $1500 will be charged. 
#(A fine will be levied every time three unique missed pickup dates occur within a six-month period for a single address.)

In [4]:
missed_pickups = df_trash[df_trash['Request'].isin(['Trash - Curbside/Alley Missed Pickup', 'Trash - Backdoor', 'Trash Collection Complaint'])]
print(f"Number of missed pickups: {len(missed_pickups)}")

Number of missed pickups: 19969


In [5]:
address_count = missed_pickups['Incident Address'].value_counts()

In [6]:
# creating sample dataframe
#data = pd.DataFrame({
#    'Date Opened': (address_count[address_count > 3] - 1)
#})

# defining rolling window
#window_size = 180

# creating rolling object
#rolling_data = data['Date Opened'].rolling(window=window_size)


# calculationg rolling mean
#rolling_mean = rolling_data.mean()
#print(rolling_mean)

In [7]:
# Make a copy of data to work with
data_q4 = missed_pickups.copy()

In [15]:
# Convert dates to datetime
data_q4['Date Opened'] = pd.to_datetime(data_q4['Date Opened'])
print(data_q4)

       Request Number Date Opened                               Request  \
4541            80284  2018-07-05            Trash Collection Complaint   
4477            79395  2018-06-29  Trash - Curbside/Alley Missed Pickup   
4511            79884  2018-07-02  Trash - Curbside/Alley Missed Pickup   
10929          155122  2019-03-15  Trash - Curbside/Alley Missed Pickup   
2214            52252  2018-03-07  Trash - Curbside/Alley Missed Pickup   
...               ...         ...                                   ...   
1851            48203  2018-02-22                      Trash - Backdoor   
2794            58920  2018-04-06  Trash - Curbside/Alley Missed Pickup   
2839            59517  2018-04-10  Trash - Curbside/Alley Missed Pickup   
3121            62051  2018-04-22  Trash - Curbside/Alley Missed Pickup   
11049          157096  2019-03-20  Trash - Curbside/Alley Missed Pickup   

                                             Description  \
4541   THERE NEEDS TO BE TASK FORCE THA

In [16]:
# Sort by address and date
data_q4 = data_q4.sort_values(['Incident Address', 'Date Opened'])
print(data_q4)

       Request Number Date Opened                               Request  \
4541            80284  2018-07-05            Trash Collection Complaint   
4477            79395  2018-06-29  Trash - Curbside/Alley Missed Pickup   
4511            79884  2018-07-02  Trash - Curbside/Alley Missed Pickup   
10929          155122  2019-03-15  Trash - Curbside/Alley Missed Pickup   
2214            52252  2018-03-07  Trash - Curbside/Alley Missed Pickup   
...               ...         ...                                   ...   
1851            48203  2018-02-22                      Trash - Backdoor   
2794            58920  2018-04-06  Trash - Curbside/Alley Missed Pickup   
2839            59517  2018-04-10  Trash - Curbside/Alley Missed Pickup   
3121            62051  2018-04-22  Trash - Curbside/Alley Missed Pickup   
11049          157096  2019-03-20  Trash - Curbside/Alley Missed Pickup   

                                             Description  \
4541   THERE NEEDS TO BE TASK FORCE THA

In [18]:
# Use rolling window to count pickups within 180 days for each address
# This looks at each pickup and counts how many pickups happened in the last 180 days
count_in_window = (
    data_q4
    .groupby('Incident Address')
    .rolling(window='180d', on='Date Opened')
    ['Date Opened']
    .count()
)
print(count_in_window)

Incident Address                                          Date Opened
, Nashville, TN , United States                           2018-07-05     1.0
1 BELLE FORREST AVE C                                     2018-06-29     1.0
10 Belle Forrest Ave                                      2018-07-02     1.0
100 Bluefield Square, Nashville, TN 37214, United States  2019-03-15     1.0
100 Braxton Hill Ct                                       2018-03-07     1.0
                                                                        ... 
compton ave & 15th ave s                                  2018-08-22     1.0
fransworth dr                                             2018-02-21     1.0
s 13th st  & woodland st                                  2018-02-23     1.0
trimble rd & lindawood dr                                 2018-06-11     1.0
xavier dr                                                 2017-12-30     1.0
Name: Date Opened, Length: 19960, dtype: float64


In [21]:
# Count how many times an address had 3+ pickups in a 180-day window
times_with_3_or_more = (count_in_window >= 3).sum()
print("3 or more pickups in 180 day window:\ times_with_3_or_more", times_with_3_or_more)


3 or more pickups in 180 day window:\ times_with_3_or_more 2181


In [23]:
# Calculate total fines: $1500 for each occurrence
total_fines_q4 = times_with_3_or_more * 1500
print(total_fines_q4)

3271500


In [13]:
#Alternative Method 2:
#This method also considers the six-month window like Alternative Method 1, but each date can only be used once to support a fine. 
#How will this difference impact the fines levied? (Example: If Jan 1st, Mar 3rd, Apr 8th, and Aug 9th were the only four dates a trash pickup was missed, 
#the original method would result in $3000 in fines [Jan, Mar, Apr, and also Mar, Apr, Aug]. However, this updated method would only result in $1500 
#because neither Mar nor Apr can be used for another fine since they were already used.)

In [24]:
# Create DataFrame
data = pd.DataFrame({
    'Date Opened': (address_count[address_count > 3] - 1)
})

# Count unique values in 'Date Opened' column using unique()
n = len(pd.unique(data['Date Opened']))

print("Number of unique values in 'Date Opened':", n)

Number of unique values in 'Date Opened': 17
